In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
from interface.TimitInterface import TimitInterface
from phoneme import initialize_model, get_needed_data

In [8]:
processor, model, device = initialize_model()

c:\Users\kavya\anaconda3\envs\accent-capstone\lib\site-packages\transformers\modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(re

In [4]:
dataset = TimitInterface()

0it [00:00, ?it/s]

Map:   0%|          | 0/2688 [00:00<?, ? examples/s]

Map:   0%|          | 0/336 [00:00<?, ? examples/s]

Map:   0%|          | 0/336 [00:00<?, ? examples/s]

In [5]:
val_data = dataset.valid
train_data = dataset.train

In [9]:
data = {}
#load data

In [6]:
train_output = list(map(lambda v: (v['audio_file'], v['word_file'], v['phonetic_file']), train_data.values()))
val_output = list(map(lambda v: (v['audio_file'], v['word_file'], v['phonetic_file']), val_data.values()))

In [9]:
# Second define your dataloader to give inputs, targets, input_length and target_length
train_dataloader = []
val_dataloader = []
for wav_path, txt_path, phn_path in train_output:
    train_dataloader.append(get_needed_data(wav_path, txt_path, phn_path))

for wav_path, txt_path, phn_path in val_output:
    val_dataloader.append(get_needed_data(wav_path, txt_path, phn_path))

ctc_loss = nn.CTCLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 1 #3
max_grad_norm = 1.0

NameError: name 'processor' is not defined

In [ ]:
for epoch in range(num_epochs):
    model.train()
    for _, _, inputs, targets, _, target_lengths in dataloader:
        optimizer.zero_grad()
        print(inputs)
        outputs = model(inputs).logits
        outputs = outputs.transpose(0, 1)
        #print(outputs.shape)
        print(outputs)
        print(outputs[0])

        if torch.isnan(outputs).any():
            print("NaN detected in outputs")
            continue
        
        input_lengths = torch.tensor([outputs.shape[0]], dtype=torch.int32)
        target_lengths = target_lengths.to(torch.int32)

        # Calculate log probabilities
        log_probs = torch.log_softmax(outputs, dim=-1)
        print(f'log_probs: {log_probs}')
        # Get predicted indices
        predicted_indices = torch.argmax(log_probs, dim=-1)  # (seq_len, batch_size)
        # Print or store the phoneme predictions
        print("Phoneme Predictions:", predicted_indices)
        
        print(targets)
        loss = ctc_loss(outputs, targets, input_lengths, target_lengths)
        print(f'loss: {loss}')
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        optimizer.step()
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for _, _, inputs, targets, _, target_lengths in val_dataloader:
            outputs = model(inputs).logits
            outputs = outputs.transpose(0, 1)
            input_lengths = torch.tensor([outputs.shape[0]], dtype=torch.int32)
            target_lengths = target_lengths.to(torch.int32)
            loss = ctc_loss(outputs, targets, input_lengths, target_lengths)
            val_loss += loss.item()
    
    val_loss /= len(val_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss:.4f}")